# RAG 基礎入門

本範例展示：
1. **第1個儲存格**：建立向量資料庫
2. **第2個儲存格**：查詢向量資料庫
3. **第3個儲存格以後**: 整合chain的功能

學習目標：理解如何將文本轉換為向量並進行相似度檢索

最後，我們會介紹如何使用 Dict 和 RunnablePassthrough 來實現相同的 RAG 流程，並且會介紹 Dict 的實際運作流程。

In [ ]:
# 第1個儲存格：建立向量資料庫

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "智慧型手機使用手冊.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db_jina")

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量資料庫...")

    # 確保文字檔案存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(
            f"檔案 {file_path} 不存在。請檢查路徑。"
        )

    # 從檔案讀取文字內容
    loader = TextLoader(file_path)
    documents = loader.load()

    # 將文件分割成塊
    # chunk_size=1000: 每個文本區塊最多 1000 個字元
    # chunk_overlap=0: 區塊之間不重疊
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print("\n--- 文件塊資訊 ---")
    print(f"文件塊數量: {len(docs)}")
    print(f"範例塊:\n{docs[0].page_content}\n")

    # 建立嵌入模型
    print("\n--- 正在建立嵌入 ---")
    print("使用 Jina Embeddings v2（繁體中文開源模型）")
    embeddings = HuggingFaceEmbeddings(
        model_name="jinaai/jina-embeddings-v2-base-zh"
    )
    print("\n--- 完成建立嵌入 ---")

    # 建立向量存儲並自動持久化
    print("\n--- 正在建立向量存儲 ---")
    
    # 使用 PersistentClient 以避免權限問題
    client = chromadb.PersistentClient(path=persistent_directory)
    
    db = Chroma.from_documents(
        docs, 
        embeddings, 
        client=client,
        collection_name="smartphone_manual"
    )
    print("\n--- 完成建立向量存儲 ---")

else:
    print("向量存儲已存在。無需初始化。")

In [ ]:
# 第2個儲存格：查詢向量資料庫

import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_jina")

# 定義嵌入模型（使用開源繁體中文模型）
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用 PersistentClient 載入現有的向量存儲
client = chromadb.PersistentClient(path=persistent_directory)

db = Chroma(
    client=client,
    collection_name="smartphone_manual",
    embedding_function=embeddings
)

# 定義使用者的問題
query = "如何設定指紋辨識？"

# 根據查詢檢索相關文件
# search_type="similarity": 使用相似度搜尋（預設方法）
# k=3: 返回最相關的 3 個文件
# 注意：由於某些 embedding 模型的相似度分數可能不在 0-1 範圍內，
#      因此使用簡單的 similarity 搜尋而非 similarity_score_threshold
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"來源: {doc.metadata.get('source', 'Unknown')}\n")

In [ ]:
# 下一個儲存格：整合 Chain - 使用 RunnableLambda 的傳統做法（和上一個 Chain 實現相同的 RAG 流程）
# 這裡的做法，其實和上一個儲存格（dict + RunnablePassthrough）功能是一樣的，只是語法不同

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_ollama import ChatOllama

question = "如何設定指紋辨識？"
print(f"問題: {question}\n")

# (步驟展示：和下一格一樣，取 2 筆相關文件)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)
retrieved_docs = retriever.invoke(question)
print(f"找到 {len(retrieved_docs)} 個相關文件\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"文件 {i} (前100字):")
    print(doc.page_content[:100] + "...\n")

print("=" * 60)
print("建立 RAG Chain（RunnableLambda 傳統做法）")
print("=" * 60)

# 和上一個儲存格一樣的 prompt 設計
template = """你是一個智慧型手機的客服助手。請根據以下參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

請用繁體中文回答，並且：
1. 只根據參考資料回答，不要編造內容
2. 如果參考資料中沒有答案，請誠實說「我在資料中找不到相關資訊」
3. 回答要清楚、具體、有條理

回答："""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOllama(model="llama3.2", temperature=0)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

def retrieve_docs(question):
    docs = retriever.invoke(question)
    return format_docs(docs)

# 注意這邊: 其實 combine_context_and_question 只是 identity mapping，和 Passthrough/dict 用法表現一致
def combine_context_and_question(inputs):
    return {"context": inputs["context"], "question": inputs["question"]}

rag_chain = (
    RunnableLambda(lambda x: {"context": retrieve_docs(x), "question": x})
    | RunnableLambda(combine_context_and_question)
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain 建立完成（RunnableLambda 寫法，跟上一格功能一樣，只是語法不同）\n")

print("=" * 60)
print("執行 RAG Chain")
print("=" * 60)
answer = rag_chain.invoke(question)

print(f"\n【AI 回答】\n{answer}\n")




In [ ]:
# 第4個儲存格：整合 Chain - 建立簡單的 RAG 問答鏈
# Chain鍊內使用dict,和RunnablePassthrough,retriever
# 觀念說明,請看下方重要觀念

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# 定義使用者的問題
question = "如何設定指紋辨識？"

print(f"問題: {question}\n")

# 步驟 1: 從向量資料庫檢索相關文件
print("=" * 60)
print("步驟 1: 從向量資料庫檢索相關文件")
print("=" * 60)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}  # 只取最相關的 2 個文件
)
retrieved_docs = retriever.invoke(question)

print(f"找到 {len(retrieved_docs)} 個相關文件\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"文件 {i} (前100字):")
    print(doc.page_content[:100] + "...\n")

# 步驟 2: 建立 RAG Chain
print("=" * 60)
print("步驟 2: 建立 RAG Chain")
print("=" * 60)

# 定義提示模板
template = """你是一個智慧型手機的客服助手。請根據以下參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

請用繁體中文回答，並且：
1. 只根據參考資料回答，不要編造內容
2. 如果參考資料中沒有答案，請誠實說「我在資料中找不到相關資訊」
3. 回答要清楚、具體、有條理

回答："""

prompt = ChatPromptTemplate.from_template(template)

# 建立 LLM（使用本地 Ollama）
llm = ChatOllama(model="llama3.2", temperature=0)

# 定義文件格式化函數
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# 建立完整的 RAG Chain
# RunnablePassthrough() 讓 question 直接傳遞下去
# retriever | format_docs 將檢索結果格式化為 context
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain 建立完成\n")

# 步驟 3: 執行 RAG Chain 並取得答案
print("=" * 60)
print("步驟 3: 執行 RAG Chain")
print("=" * 60)

answer = rag_chain.invoke(question)

print(f"\n【AI 回答】\n{answer}\n")



## 📚 重要觀念：理解 Dict 在 LangChain 中的用法

### 🎯 為什麼 RAG Chain 內使用 Dict？

在 RAG（檢索增強生成）系統中，我們需要同時處理**兩個不同的資料流**：

1. **使用者問題**（question）- 需要原封不動地傳遞給 LLM
2. **檢索到的上下文**（context）- 需要經過格式化處理後傳遞給 LLM

### 🔧 Dict 的核心作用

**Dict 就像一個「資料整合器」**，它能夠：

```python
# Dict 的結構化處理方式
{
    "context": retriever | format_docs,    # 處理檢索到的文件
    "question": RunnablePassthrough()      # 直接傳遞原始問題
}
```

### 📝 實際範例說明

讓我們看看您筆記本中的實際程式碼：

```4:4:4_rag/1_rag_basics.ipynb
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
```

### 🔍 詳細運作流程

1. **輸入階段**：
   - 使用者問題：`"如何設定指紋辨識？"`

2. **Dict 處理階段**：
   - `"context"` 鍵：執行 `retriever | format_docs`
     - 檢索相關文件
     - 格式化為字串
   - `"question"` 鍵：執行 `RunnablePassthrough()`
     - 直接傳遞原始問題

3. **輸出階段**：
   ```python
   {
       "context": "iPhone 15 有指紋辨識功能，設定步驟：1. 進入設定 2. 選擇 Touch ID 3. 添加指紋...",
       "question": "如何設定指紋辨識？"
   }
   ```

### 💡 為什麼不使用其他方法？

**❌ 如果不用 Dict，會遇到的問題：**

```python
# 錯誤做法：無法同時處理兩個資料流
def wrong_approach(question):
    context = retriever.invoke(question)  # 只能處理一個流程
    return context  # 問題資訊丟失了
```

**✅ 使用 Dict 的優勢：**

1. **並行處理**：同時處理多個資料流
2. **結構清晰**：明確的鍵值對對應關係
3. **可擴展性**：容易添加新的欄位
4. **類型安全**：每個欄位都有明確的用途

### 🎨 與 Prompt Template 的完美配合

Dict 的輸出會自動對應到 Prompt Template 中的變數：

```python
template = """
參考資料：{context}    # ← 對應 Dict 中的 "context" 鍵
使用者問題：{question}  # ← 對應 Dict 中的 "question" 鍵
"""
```

### 🚀 實際應用場景

Dict 在 RAG 中的應用不僅限於基本的問題-上下文配對，還可以擴展到：

```python
# 擴展的 Dict 用法
{
    "context": retriever | format_docs,
    "question": RunnablePassthrough(),
    "user_id": lambda x: get_user_id(x),      # 新增：使用者 ID
    "timestamp": lambda x: get_timestamp(),   # 新增：時間戳
    "conversation_history": lambda x: get_history(x)  # 新增：對話歷史
}
```

### 📋 總結

**Dict 在 RAG Chain 中的核心價值：**

1. **資料整合**：將多個處理流程的結果整合在一起
2. **流程控制**：讓不同的資料流能夠並行處理
3. **結構化管理**：提供清晰的鍵值對對應關係
4. **擴展性**：容易添加新的處理流程

LangChain 的 RAG 實作中，Dict 是不可或缺的重要組件！它讓我們能夠優雅地處理複雜的多資料流場景，同時保持程式碼的清晰和可維護性。


## 介紹retriever
- 為什麼retriever可以放在chain內,
- 為什麼會自動執行invoke()
- 傳出來的是什麼?

## 🔍 Retriever 深度解析

### 🎯 什麼是 Retriever？

**Retriever（檢索器）** 是 RAG 系統中的核心組件，負責從向量資料庫中檢索與使用者問題相關的文件。

### 🔧 為什麼 Retriever 可以放在 Chain 內？

#### 1. **Runnable 介面實現**

- Retriever 實現了 LangChain 的 Runnable 介面
- format_docs（一個普通函數）會自動包裝成 RunnableLambda
- RunnablePassthrough() 本身就是一個 Runnable
- ChatPromptTemplate 是 Runnable
- LLM 也是 Runnable

Retriever 實現了 LangChain 的 `Runnable` 介面，這意味著它可以：

```python
# Retriever 是一個 Runnable 物件
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)

# 可以像其他 Runnable 一樣使用 | 操作符
retriever | format_docs
```

#### 2. **自動 Chain 整合**

當 retriever 放在 Chain 中時，LangChain 會自動處理：

```python
# 在 Chain 中的使用方式
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
```

**內部運作流程：**
1. 輸入問題 → retriever
2. retriever 執行檢索 → 返回 Document 列表
3. Document 列表 → format_docs 函數
4. 格式化後的文字 → 傳遞給 prompt

### ⚡ 為什麼會自動執行 invoke()？

#### 1. **LCEL（LangChain Expression Language）機制**

當 Chain 執行時，LangChain 會自動調用每個組件的 `invoke()` 方法：

```python
# 當你執行
result = rag_chain.invoke("如何設定指紋辨識？")

# LangChain 內部會自動執行：
# 1. retriever.invoke("如何設定指紋辨識？")
# 2. format_docs(retriever_result)
# 3. prompt.invoke({"context": formatted_docs, "question": "如何設定指紋辨識？"})
# 4. llm.invoke(prompt_result)
# 5. StrOutputParser().invoke(llm_result)
```

#### 2. **自動化流程控制**

```python
# 手動執行 vs 自動執行
# 手動方式：
question = "如何設定指紋辨識？"
docs = retriever.invoke(question)  # 手動調用
formatted_context = format_docs(docs)
result = prompt.invoke({"context": formatted_context, "question": question})

# Chain 自動方式：
result = rag_chain.invoke(question)  # 自動調用所有組件的 invoke()
```

### 📤 Retriever 傳出來的是什麼？

#### 1. **Document 物件列表**

Retriever 返回的是 `Document` 物件的列表：

```python
# retriever.invoke() 的返回值
retrieved_docs = retriever.invoke("如何設定指紋辨識？")

print(f"類型：{type(retrieved_docs)}")  # <class 'list'>
print(f"長度：{len(retrieved_docs)}")   # 2

# 每個 Document 物件的結構
for i, doc in enumerate(retrieved_docs):
    print(f"文件 {i+1}:")
    print(f"  內容：{doc.page_content[:100]}...")
    print(f"  元數據：{doc.metadata}")
    print(f"  類型：{type(doc)}")  # <class 'langchain_core.documents.Document'>
```

#### 2. **Document 物件的詳細結構**

```python
# Document 物件包含：
class Document:
    page_content: str      # 文件的主要內容
    metadata: dict         # 元數據（來源、頁碼等）
    
# 實際範例：
doc = retrieved_docs[0]
print(f"內容：{doc.page_content}")
print(f"元數據：{doc.metadata}")
# 輸出：
# 內容：iPhone 15 有指紋辨識功能，設定步驟：1. 進入設定 2. 選擇 Touch ID 3. 添加指紋...
# 元數據：{'source': '/path/to/smartphone_manual.txt'}
```

#### 3. **在 Chain 中的轉換過程**

```python
# 完整的資料流轉換
def format_docs(docs):
    """將 Document 列表轉換為字串"""
    return "\n\n".join([doc.page_content for doc in docs])

# Chain 中的處理流程：
# 1. retriever.invoke(question) → [Document, Document, ...]
# 2. format_docs(docs) → "文件1內容\n\n文件2內容\n\n..."
# 3. 傳遞給 prompt 的 {context} 變數
```

### 🔄 實際運作範例

讓我展示一個完整的 retriever 運作過程：

```python
# 步驟 1：建立 retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)

# 步驟 2：手動執行（了解內部運作）
question = "如何設定指紋辨識？"
print(f"問題：{question}")

# 檢索過程
docs = retriever.invoke(question)
print(f"\n檢索結果類型：{type(docs)}")
print(f"檢索到 {len(docs)} 個文件")

# 查看每個 Document
for i, doc in enumerate(docs, 1):
    print(f"\n文件 {i}:")
    print(f"  內容長度：{len(doc.page_content)} 字元")
    print(f"  前100字：{doc.page_content[:100]}...")
    print(f"  元數據：{doc.metadata}")

# 步驟 3：格式化處理
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

formatted_context = format_docs(docs)
print(f"\n格式化後的上下文長度：{len(formatted_context)} 字元")
print(f"格式化結果：{formatted_context[:200]}...")
```

### 📋 總結

**Retriever 的核心特性：**

1. **Runnable 介面**：可以無縫整合到 LangChain Chain 中
2. **自動 invoke()**：LCEL 機制自動調用檢索功能
3. **Document 列表輸出**：返回結構化的文件物件列表
4. **元數據保留**：保持文件的來源和相關資訊
5. **可配置性**：支援不同的檢索策略和參數

**在 RAG Chain 中的作用：**
- **輸入**：使用者問題（字串）
- **處理**：向量相似度檢索
- **輸出**：相關 Document 物件列表
- **整合**：與其他 Chain 組件無縫串接

這就是為什麼 retriever 能夠如此優雅地整合到 LangChain 的 Chain 架構中，成為 RAG 系統的核心組件！